<center><h1>Máquina de café com autômatos</h1></center> 

**Estados possíveis:**

|Estado|Descrição|
|:----:|:-------:|
|q0|Esperando seleção|
|q1|Esperando pagamento|
|q2|Preparando café|
|q3|Entregando café|
|q4|Devolvendo troco (se aplicável)|
|q5|Operação cancelada|

**Alfabeto**

|Símbolo|Descrição|
|:-----:|:-------:|
|0|Falso|
|1|Verdadeiro|

**Tipos de bebidas**

| Bebida          | Preço Venda(Reais) | Preço Custo(Reais)|
|:---------------:|:----------------:|:----------------:|
| Café filtrado   | 7.50             | 3.00             |
| Café latte      | 8.00             | 4.00             |
| Café frappé     | 14.50            | 7.25             |
| Chocolate quente| 9.50             | 5.00             |
| Chá             | 6.50             | 2.00             |

### Importações

In [3]:
from automathon import DFA

#### Definição do problema

Modelando o sistema da máquina de café:

- Cada opção disponibilizada pelo sistema possui um custo de produção atrelado, discriminado na tabela de tipos de bebidas vista acima
- O sistema conta com um caixa, que é verificado no automato para conferir a possibilidade de entrega do pedido
- O cliente deve inserir um preço maior ou igual ao correspondente da bebida escolhida. Se o pagamento inserido for maior que o necessário, o sistema volta um troco para ele

In [147]:
class CoffeMachine:
    def __init__(self):
        
        self.caixa = 1000
        
        self.opcoes = {
            'Café filtrado': (7.50, 3),
            'Café late': (8, 3.50),
            'Café frappé': (14.50, 4.25),
            'Chocolate quente': (9.50, 5),
            'Chá': (6.50, 2)
        }
        
        self.Q = {'q0', 'q1', 'q2', 'q3', 'q4', 'q5'}
        self.sigma = {'0', '1'}
        self.delta = {
            'q0': {'0': 'q5', '1': 'q1'},
            'q1': {'0': 'q5', '1': 'q2'},
            'q2': {'0': 'q2', '1': 'q3'},
            'q3': {'0': 'q3', '1': 'q4'},
            'q4': {'0': 'q4', '1': 'q4'},
            'q5': {'0': 'q5', '1': 'q5'}
        }
        self.initial_state = 'q0'
        self.F = {'q3', 'q4'}
        
        self.automata = DFA(self.Q, self.sigma, self.delta, self.initial_state, self.F)
        
    def exibe_opcoes(self):
        i = 0
        print('Número - Opção: Preço')
        for key, item in self.opcoes.items():
            print(f'{i} - {key}: {item[0]}')
            i+=1
        print('')
        
    def check_pagamento(self, cliente):
        return self.opcoes.get(cliente.opcao_desejada)[0] <= cliente.pagamento
        
    def get_troco(self, cliente):
        return cliente.pagamento - self.opcoes.get(cliente.opcao_desejada)[0]
    
    def check_caixa(self, cliente):
        return self.caixa > (cliente.pagamento + self.opcoes.get(cliente.opcao_desejada)[1] + self.get_troco(cliente))
    
    def att_caixa(self):
        self.caixa = self.caixa + cliente.pagamento - self.opcoes.get(cliente.opcao_desejada)[1] - self.get_troco(cliente)
    
    def exibe_caixa(self):
        print(f'O caixa está em: {self.caixa}')
    
    def atualiza_caixa(self, cliente):
        self.caixa = self.caixa + cliente.pagamento - self.opcoes.get(cliente.opcao_desejada)[1]
        
    def check_automato(self, cliente):
        
        entrada = '1' + str(int(self.check_pagamento(cliente))) + str(int(self.check_caixa(cliente))) + '1' + str(int(self.get_troco(cliente)>0))
        
        if self.automata.accept(entrada):
            print('Pedido aprovado!')
            self.att_caixa()
        else:
            print('Pedido não aprovado')
        
        if not self.check_caixa(cliente):
            print('Caixa insuficiente!')
        
        cliente.set_troco(self.get_troco(cliente))
        

In [139]:
class Cliente:
    def __init__(self, opcao_desejada, pagamento):
        self.opcao_desejada = opcao_desejada
        self.pagamento = pagamento
        
    def set_troco(self, troco):
        self.troco = troco
        
    def exibe_troco(self):
        if self.troco > 0:
            print(f'Seu troco é de: {self.troco}')

#### instanciando o sistema da máquina de café

In [148]:
maq = CoffeMachine()

#### Checando se o autômato gerado é válido para um AFD

In [136]:
maq.automata.is_valid()

True

#### Visualização do autômato

In [41]:
# estilo default
maq.automata.view("DFA Visualization")


In [42]:
maq.automata.view(
    file_name="DFA Visualization_personalizado",
    node_attr={'fontsize': '20'},
    edge_attr={'fontsize': '20pt'}
)

![Automato que representa a máquina de café](./DFA_Visualization_personalizado.png)

#### Funcionamento dos pedidos

- As opções são exibidas ao usuário
- O usuário seleciona um item do cardápio e insere o seu pagamento
- De acordo com as informações sugeridas, o sistema, por meio do AFD, verifica a possibilidade de conclusão do pedido
- Caso aplicável, o troco é devolvido ao cliente
- Caso quiser parar de utilizar o sistema, basta entrar com -1 na seleção de opções

In [150]:
print('Entre com -1 para sair\n')
maq.exibe_opcoes()

while True:
    opcao = int(input('Selecione o número da opção desejada: '))
    
    if opcao < 0:
        print('Obrigado por ser nosso cliente! Até mais!')
        break
    if opcao > 4:
        print('Não há essa opção no cardápio\n')
        continue
        
    pagamento = float(input('Entre com o pagamento: '))

    opcao_desejada = list(maq.opcoes.keys())[opcao]

    cliente = Cliente(opcao_desejada, pagamento)

    resultado = maq.check_automato(cliente)
    cliente.exibe_troco()
    maq.exibe_caixa()
    
    print()
    

Entre com -1 para sair

Número - Opção: Preço
0 - Café filtrado: 7.5
1 - Café late: 8
2 - Café frappé: 14.5
3 - Chocolate quente: 9.5
4 - Chá: 6.5

Selecione o número da opção desejada: 5
Não há essa opção no cardápio

Selecione o número da opção desejada: -1
Obrigado por ser nosso cliente! Até mais!
